## This notebook runs inference on a GEARS model trained on Norman

- Download trained GEARS model and Dataloader from Dataverse
- Model is trained on Norman et al. 2019 (Science) dataset
- Example below showing how to make perturbation outcome prediction and GI prediction

In [1]:
import sys
sys.path.append('../')

from gears import PertData, GEARS
from gears.utils import dataverse_download
from zipfile import ZipFile 
from tqdm import tqdm

/home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/libpyg.so)
  import torch_geometric.typing
/home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_scatter/_version_cuda.so)
  import torch_geometric.typing
/home/c22200541/miniconda3/envs/gears/lib/python3.11/site-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libc

In [ ]:
'''## Download dataloader from dataverse
dataverse_download('https://dataverse.harvard.edu/api/access/datafile/6979957', 'norman_umi_go.tar.gz')

## Extract and set up dataloader directory
import tarfile
with tarfile.open('norman_umi_go.tar.gz', 'r:gz') as tar:
    tar.extractall()'''

In [3]:
'''## Download model from dataverse
dataverse_download('https://dataverse.harvard.edu/api/access/datafile/10457098', 'model.zip')

## Extract and set up model directory
with ZipFile(('model.zip'), 'r') as zip:
    zip.extractall(path = './')'''

Found local copy...


### Load model and dataloader

In [2]:
pert_data = PertData('./data')
pert_data.load(data_name = 'norman')
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 32)

gears_model = GEARS(pert_data, device = 'cuda:0', 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')
#gears_model.load_pretrained('./model_ckpt')
gears_model.load_pretrained('test_model')

Found local copy...
Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['RHOXF2BB+ctrl' 'LYL1+IER5L' 'ctrl+IER5L' 'KIAA1804+ctrl' 'IER5L+ctrl'
 'RHOXF2BB+ZBTB25' 'RHOXF2BB+SET']
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at ./data/norman/splits/norman_simulation_1_0.75.pkl
Simulation split test composition:
combo_seen0:9
combo_seen1:43
combo_seen2:19
unseen_single:36
Done!
Creating dataloaders....
Done!


### Make transcriptional outcome predictions

In [3]:
test_adata = pert_data.adata[pert_data.adata.obs['split'] == 'test']
unique_conds = list(set(test_adata.obs['condition'].astype(str).unique()) - set(['ctrl']))
pred = []
for condition in unique_conds:
    gene_list = condition.split('+')
    if 'ctrl' in gene_list:
        gene_list.remove('ctrl')
    gears_pred = gears_model.predict([gene_list])
    pred.append(gears_pred)

### Make GI outcome prediction

In [8]:
## If reproducing results from paper, you can use the same gene set, 
## although the function works even if GI_genes_file is set to None

dataverse_download('https://dataverse.harvard.edu/api/access/datafile/6979958', 
                   'genes_with_hi_mean.npy')

gears_model.GI_predict(['CNN1', 'CBL'], GI_genes_file='./genes_with_hi_mean.npy')

Downloading...
100%|███████████████████████████████████████| 45.0k/45.0k [00:00<00:00, 299kiB/s]


{'ts': TheilSenRegressor(fit_intercept=False, max_iter=1000,
                   max_subpopulation=100000.0, random_state=1000),
 'c1': 0.7503545666280899,
 'c2': 1.024057001878076,
 'mag': 1.2695372065265131,
 'dcor': 0.8333099467500071,
 'dcor_singles': 0.7438353675702762,
 'dcor_first': 0.7742488397309704,
 'dcor_second': 0.795405078049431,
 'corr_fit': 0.9218557390881672,
 'dominance': 0.1350576013768744,
 'eq_contr': 0.9734019320440574}